## Connecting our Research Question to Data

***Does the appearance of JHU in the movies affect application rates?***

What data will we need to answer this question?

- List of JHU appearances in film
    - Year of appearance
    - Characteristics of the movie - genre, rating, etc.
- JHU application rates
    - Year of the application rate

## Finding and accessing the data we need 

This is usually the hard part, but as a Love Data Week present we've pre-prepared a set of files that can help us answer our research question ❤️

- **`jhu-titles.csv`**: Titles and Years of titles featuring, or filmed at, JHU, scraped from [Johns Hopkins on Film](https://hub.jhu.edu/2015/10/13/johns-hopkins-on-film/)
- **`imdb-title-basics.csv`**: A truncated version of `title.basics.tsv` from [IMDB Datasets](https://datasets.imdbws.com/) including 26 titles that match the Title/Year combination from `jhu-titles.csv`
- **`imdb-ratings.csv`**: The original `title.ratings.tsv` from [IMDB Datasets](https://datasets.imdbws.com/), exported as a .csv
- **`IPEDS_Raw`**: folder of 20 files, each containing JHU application and acceptance data from one year 2001 - 2020, downloaded from [IPEDS](https://nces.ed.gov/ipeds) (Integrated Postsecondary Education Data System)

## Data cleaning considerations

While we have pre-prepared files that we can use to answer our research question, they are not 100% ready. We do have some data considerations to think about before our data will be ready to answer our research question:

- There are often more than one match in `imdb-title-basics.csv` to the titles listed in `jhu-titles.csv`, so we will have to be careful to ensure we are selecting the right title.
- Titles are not always consistent, for example **The Curve (1998)** has the original title **Dead Man's Curve**, so we will need to be careful with our titles during analysis.
-  Both `imdb-title-basics.csv` and `jhu-titles.csv` have fields with year information, which we will need to ensure is treated as a date, and not an as arbirtrary numeric value.
- Finally, we have data spread across many many files! We have a little bit of work ahead of us to ensure these are combined in a way that makes them meaningful and useful.

## Preparing Acceptance Rate Data

Prior to the workshop, we downloaded data on applications and acceptances from IPEDS. 

Let's go over to the [IPEDS Custom Data File page](https://nces.ed.gov/ipeds/datacenter/InstitutionByName.aspx?goToReportId=5&sid=506bb19e-f71f-4863-b838-95a0b656aebf&rtid=5) and walk through the process.

Key takeaways:
- Our earliest JHU film appearance is 1946,  but we found out that acceptance data is only available from 2001 on, so we'll have to filter out films from prior to 2000 when plotting the data.
- Each year downloaded as a separate file. Looks like we'll have to find a way to combine them.

Let's take a look at a couple of the files we downloaded.

We'll be using the a Python library called `pandas` to read and prepare this data - to read in a csv to a spreadsheet-like structure called a DataFrame, we'll use the `pandas` function [`read_csv()`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)

In [ ]:
import pandas as pd 
pd.read_csv("IPEDS_Raw/CSV_192026-1007.csv").head()

In [ ]:
pd.read_csv("IPEDS_Raw/CSV_192026-1030.csv").head()

We can see that each csv has a single row representing a single year, and the variables we downloaded from IPEDs. But the variable names have different prefixes in each file - we'll have to address this when combining them.

## Combining IPEDS files

To combine our files, we'll use the [`concat()`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) function from pandas, which allows us to append either columns or rows from one DataFrame to another. The axis argument determines whether we 'stack' two DataFrames on top of each other or side by side.

axis=0 refers to the rows of the DataFrames, and tells pandas to stack the DataFrames vertically. axis=1 refers to the columns of the DataFrames and tells pandas to stack the DataFrames horizontally.

Because we want to stack vertically, we need to make sure the two DataFrames have the same columns and respective data types.

Before we can use `concat()`, we need to read all our files into dataframes. As we saw above, to use `read_csv()` we need the path to the file. While  we could copy these out manually, Python has a method called `glob()` for listing the contents of a given directory.

In [ ]:
import glob
import os

# Define the directory path where your CSV files are located. Use a relative path (e.g., './my_csv_files') or an absolute path
directory_path = './IPEDS_Raw'

# Get a list of all CSV files in the directory. The pattern '*.csv' matches all files ending with .csv
ipeds_files = glob.glob(os.path.join(directory_path, '*.csv'))
print(ipeds_files)

Now, we can loop through all of the files in the directory we designated above and read each in as a pandas dataframe.
To make sure each dataframe has the same column names, we'll use the `header` and `names` arguments of `read_csv()` to set standard column names manually as we read each file in.
Then, we'll use `concat()` to combine all of the dataframes together into a dataframe called `combined_df`

In [ ]:
# Create a empty dataframe to insert the data from our csvs into
ipeds_df = pd.DataFrame()

for file in ipeds_files:
    df = pd.read_csv(file, header = 0, names = ['id', 'institution', 'year', 'applicants_total', 'applicants_men', 'applicants_women',
                                                'admissions_total', 'admissions_men', 'admissions_women', 'enrolled_total', 'enrolled_men',
                                                'enrolled_women', 'enrolled_ft_total', 'enrolled_ft_men', 'enrolled_ft_women',
                                                'enrolled_pt_total', 'enrolled_pt_men', 'enrolled_pt_women'])
    ipeds_df = pd.concat([df, ipeds_df], ignore_index=True)
ipeds_df.head(20)

## Planning our data preparation

Now that all our data is in one place, let's recall our goal: to generate a table with acceptance rate by year. We can use the pandas function [`describe()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html#pandas.DataFrame.describe) to generate a quick summary of our data to see if we want to add anything else.

In [ ]:
# use df_name.describe() to look at summary stats


## Handling missing data

We can calculate the overall acceptance rate from `admissions_total` and `applicants_total`. It looks like we could also break down acceptance rate by gender and full-time status. But weird - the `enrolled_pt` variables have a count of 13, while the others have 20. This is because `count()` doesn't count `NaN` values - so these columns have some missing data. 

Another way to check for missing data is with the pandas function [`isna()`](https://pandas.pydata.org/docs/reference/api/pandas.isna.html)

In [ ]:
# check for missing values with df.isna()


When applied to a DataFrame, `is_na()` returns a DataFrame with the same dimensions with True/False values indicating whether that value is missing. To get see which variables have missing data at a glance, we can *chain* the pandas functions [`any()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.any.html#pandas.Series.any) or [`mean()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.mean.html#pandas.Series.mean) onto `combined_df.isna()`. 

In [ ]:
# Chain any() onto df.isna() to see columns with missing values


In [ ]:
# Chain mean() onto df.isna() to see percentage of missing data in each column


Since the part-time variables are missing a sufficient amount of data, looking at acceptance rates for full-time status won't be too meaningful. But since the other variables are complete, let's go ahead and use the pandas function [`dropna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html#pandas.DataFrame.dropna) to remove columns with missing data. Remember from `concat()`, setting the `axis` argument = 1 specifies that we want to drop columns. specifying axis = 0 would remove rows with missing values - 35% of our dataframe with good data in most columns!

The `inplace` argument applies the change to our original dataframe.

In [ ]:
# use df.dropna() to remove missing values in rows or columns


## Calculating the acceptance rate

Finally, let's calculate the variables for acceptance rate.

The syntax to create a new column in pandas is to assign some value (in this case, the quotient of two other columns) to `df_name[new_column_name]`

In [ ]:
# create acceptance rate variables for overall, men, and women


## Finalizing the dataframe

We only need some of these variables for our analysis. To keep things clean, let's just keep `year`, `acceptance_rate`, `acceptance_rate_men`, and `acceptance_rate_women`. 

The syntax for subsetting in pandas is df[column name], where column_name can also be a list of columns.

In [ ]:
# create a new dataframe called acceptance_rates with only the year and acceptance rate variable 


## Finalizing the dataframe

We can sort the dataframe with the function [`sort_values()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.sort_values.html#pandas.Series.sort_values), with the `by` argument set to the column we want to sort by, `inplace` set to true to modify the original dataframe, and `ignore_index` set to true to re-number the rows in our new order.

In [ ]:
# use df.sortvalues() to sort the new dataframe by year


## Preparing JHU titles

We want to explore titles filmed at, or about, Johns Hopkins, as described in the HUB article [Johns Hopkins on Film](https://hub.jhu.edu/2015/10/13/johns-hopkins-on-film/), and combine it with title information from IMDB:

<div> <img src="images/hopkins-on-film.png" width="500"/> </div>


### Load JHU and IMDB data
First we will read in our JHU titles, by importing `jhu-titles.csv`. Then let's read in our IMDb data. First we will start with basic information about the titles, by importing the `imdb-title-basics.csv` file:

In [31]:
# Load in the IMDb and JHU title data


From the IPEDS data, you remember we used the convention `variable_name` for columns, where the variable titles were lowercase, and separated by spaces for multi-word variables. This is otherwise known as `snake_case` 🐍. Let's do the same here:

In [30]:
# Explore the variable names


### Clean our variable names
We see that our variables are in camelCase and we want to convert them to snake_case. We can write a little helper to do that:


In [29]:
# Write a short function to convert camel case to snake case


What's happening in this little function?

Remember our goal is to convert `titleType` to `title_type`. Our function takes a string, for example `titleType` as an argument, and returns the snake case format of that string.

- We use Python's `re` module, which stands for "regular expression". Regular Expressions (or Regex for short) provide us a systematic way of finding text matches in a string, and lets us do things with those matches.
  
- We can use the `re.sub()` function which can replace a match, with a new value. For example, `re.sub('apple', 'banana', 'apple bread')` would replace the word "apple" with the word "banana" in `apple bread` so we end up with `banana bread` (yum!)
- In our function, `r([A-Z])` matches any capital letter (in our case `T`). The `r` prefix indicates we are using a regular expression. The `[A-Z]` says find any upper case between the letters A to Z (so all of them). By putting everything between parentheses `([A-Z])` that says that if we find something, we want to keep it for later. This is called a "capturing group".
- Our second argument is the replacement. `_\1` replaces the `T` with an underscore + the same letter it found (`\1` is the "capturing group, and it contains our first match), so we end up with `title_Type`.
- Now we can use the `lower.()` function to return the result in lowercase: `title_type`

In [28]:
# Convert imdb_titles to snake_case using our camel_to_snake function


### Merge JHU title data with IMDb data¶
Now let's try to bring our data together! We would like add the variables from `imdb_titles` to `jhu_titles` when an IMDb title matches a title in `jhu_titles`.

We can do this using a `pandas` function - `pd.merge()`. We are joining on multiple columns (title and year). In SQL this is known as a "compound key".

We use a left join into `jhu_titles`. This means that:

- We keep every row from the left table (`jhu_titles`).
- If you can find matching rows in the right table (`imdb_titles`), attach them.
- If you can’t, keep the row anyway and fill the right side with NaNs.

There are two valid title columns in `imdb_titles` to choose: `original_title` and `primary_title`. Let's start with `original_title` and see how we do.

In [27]:
# Join jhu_titles and imdb_titles on title into jhu_imdb


That looks good! Ok now let's add the ratings to our `jhu_imdb` dataframe. Both our `jhu_imdb` and `imdb_ratings` dataframes contain a uniquely identifying key: `tconst`. Each title has a unique `tconst`. We will again use a `pd.merge` to join these two dataframes on our unique key. In SQL this is called a "primary key". If `tconst` was our index (instead of the integer 0, 1, 2, 3, ... labeling each row), then we would use a pandas join, but since it's a regular column in both, then we will use `pd.merge`.

In [26]:
# Join jhu_imdb and imdb_ratings using pandas merge


That looks great, but there are a few data quality issues. Can you see them?

### Data cleaning JHU titles

In [25]:
# Check the data types for jhu_imdb_ratings to ensure consistency


- `year` is an int64
- `start_year` and `end_year` are float64
- No year variable is in a proper date-time format!
- Votes are a count, so they shound be an integer – we don't expect half votes.
- Our genre isn't tidy, we have more than once genre listed in a single field.

In [24]:
# Convert our years to a numeric


This is a good time to check the number of rows in our dataframe. Do we have as many rows as we have titles in `jhu_titles`? More? Less?

In [23]:
# Compare jhu_titles and jhu_imdb_ratings length


Looks like we have more rows than we have titles, so we need to do some filtering. Sometimes, we can't automate everything. Here we need to rely on our knowledge of the titles to filter out rows that don't match. Let's take a look at the complete dataframe, and then strategize about how to filter out rows that don't correspond to a title in jhu_titles.


In [22]:
# Explore jhu_imdb_ratings


Just by looking at the dataframe, a few things stand out:

- "Veep" and "House of Cards" are TV Series, so we don't need individual episodes, just the IMDB entry for the series.
- We know that "The Social Network", "The Invasion", and "Head of State" are all movies.

With that information alone, we can filter out our extraneous rows:

In [21]:
# Filter out individual episodes and titles non-movie titles for The Social Network,
# The Invasion, and Head of State


We have now filtered our data down to the 13 titles listed in `jhu_titles`! The last thing to do is to ensure our data is tidy, that is:
- One observation per row,
- One variable per column,
- One value per cell.

We see that's not the case, `genre` has one or more values per cell. Let's clean that up!

We are going to chain our data cleaning using the `.` operator, since we have a number of steps.
1. We create a new column (`genre`) and assign it the values of `genres` split into a list of strings.
2. We `explode` genre, which means we create a new row for each list element present in `genre`, and copy the other variables into that new row.
3. Since our genres were originally in a single string, like: `Comedy, Drama, Sci-Fi`, when we split them, "Drama" and "Sci-Fi" still had a space in front of them, and we want to remove that using `str.strip()`. Do operate on each row, and remove the `genre` spaces for that row, we write an anonymous function, otherwise know as a lambda expression. This is an unnamed function, since we only need to use it once, and don't need to preserve it for use elsewhere.
4. Since we have duplicate rows, our index might look something like: `[0, 1, 1, 2, 2, 2, 3, 3, 4, 5, 5]`, so we need to reset it so that the index increments by one for each row. 

In [20]:
# Split each list of genres, by title, into individual rows


## Visualizing our research question:

> Does the appearance of JHU in the movies affect acceptance rates?

Before we can begin visualizing our data, we have a little data cleaning left. We know we don't have acceptance_rates for every title.

In [18]:
# Find earliest acceptance rate in acceptance_rates


We need to filter our data down to titles that were released on years we have acceptance_rate data.

In [19]:
# Filter the rows of jhu_clean to years that we have acceptance_rate data


Now we can begin to visualize our research question. There are many approaches we could take, and there is no right answer as to which would be the most effective. We will demonstrate one pathway, of many. 

We need to display JHU's annual acceptance rates, and then display the titles that were released during those years, overlayed, so we can draw a comparison between acceptance rates and the titles (and attributes of the titles).

We have a little data cleaning left, however. We need to convert acceptance_rates from wide to long format. Let's take a look and see what that means:

In [16]:
# Explore acceptance_rates


Each acceptance rates for women, men, and overall each have their own column. In this way, the dataframe is wide (the variables are spread across subsequent columns). We instead want a single column for acceptance rates, and a single column indicating the class (women, men, or overall). When we spread our variables over subsequent rows, our dataframe is "long".

We can do this using the pandas function `melt`:

In [17]:
# Convert acceptance_rates from wide to long using pd.melt


Now we can begin by visualizing acceptance rates by year:

In [ ]:
# Create a lineplot of acceptance rates by year

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.ticker import MultipleLocator

palette = {'Overall': (.2, .2, .2, .9),
           'Women': (.2, .2, .8, .2),
           'Men': (.2, .8, .2, .2)}


### Display titles with acceptance rates
If we want to plot the titles, we need a shared y-axis attribute, in this case acceptance_rate. We need merge our acceptance rates into our `jhu_title_current` dataframe:


In [ ]:
# Merge jhu_titles_current and acceptance_rates


In [ ]:
# Display titles

palette = {'Overall': (.2, .2, .2, .9),
           'Women': (.2, .2, .8, .2),
           'Men': (.2, .8, .2, .2)}

title_type_palette = {
    'movie': '#3498DB',        
    'tvSeries': '#E67E22',     
    'tvMiniSeries': '#9B59B6', 
    'tvMovie': '#1ABC9C'       
}


It seems like there might be a (spurious) correlation between title type and acceptance rate. Let's take a look:

In [ ]:
# Compute average overall acceptance rate, by title type
